In [ ]:
import adcUtil  as adc
import math
import RPi.GPIO as GPIO
import time
import numpy as np
import pigpio

# Reads the cartesian value of the left stick from ADC CE0
def getStickL():
    xpos = adc.baseADC0(channel=1)
    ypos = adc.baseADC0(channel=0)
    # -511.5 to center output on 0, from -511 to 511
    return (xpos-511.5), (ypos-511.5)

# Reads the cartesian value of the right stick from ADC CE1
def getStickR():
    xpos = adc.baseADC1(channel=1)
    ypos = adc.baseADC1(channel=0)
    return (xpos-511.5), (ypos-511.5)

# With a radius and theta value, press the right key(s)
def getStickKey(r,theta):
    
    # If the analog stick is near the middle/neutral position, do nothing
    if (r < 100): 
        releaseAll();
        return;
    
    # W D
    if (theta > 30 and theta < 60):
        releaseAll();
        keyboard.press('w')
        keyboard.press('d')   
        
    # W
    elif (theta > 60 and theta < 120):
        releaseAll();        
        keyboard.press('w') 
        
    # W A
    elif (theta > 120 and theta < 150):
        releaseAll();
        keyboard.press('w')
        keyboard.press('a') 
        
    # A
    elif (theta > 150 and theta < 210):
        releaseAll();        
        keyboard.press('a')  
        
    # A S
    elif (theta > 210 and theta < 240):
        releaseAll();
        keyboard.press('a')
        keyboard.press('s') 
        
    # S
    elif (theta > 240 and theta < 300):
        releaseAll();        
        keyboard.press('s')
        
    # S D
    elif (theta > 300 and theta < 330):
        releaseAll();
        keyboard.press('s')
        keyboard.press('d') 
        
    # D
    elif (theta > 330 or (theta) < 30):
        releaseAll();        
        keyboard.press('d') 
    else:
        return;    

# Catch all function to release every key to simulate holding down keys
def releaseAll():
    keyboard.release('w')
    keyboard.release('a')
    keyboard.release('s')
    keyboard.release('d')
    
# Return the theta when given cartesian x y
def angle(dx, dy):    
    rads = math.atan2(dy, dx)
    rads %= 2 * math.pi
    return math.degrees(rads)

# Return radius when given cartesian x y
def radi(dx, dy):
    r = math.sqrt(dx ** 2 + dy ** 2)
    return r

# Beeps a unique note per button when it's pressed
buzPin = 13
def beep(button):        
    # Sleep length, small value for short beep
    d = .05
        
    if button == 1:
        pi.hardware_PWM(buzPin, 261, int(.5e6))
        time.sleep(d)
        pi.hardware_PWM(buzPin, 0, int(.5e6))
            
    if button == 2:
        pi.hardware_PWM(buzPin, 291, int(.5e6))
        time.sleep(d)
        pi.hardware_PWM(buzPin, 0, int(.5e6))
        
    if button == 3:
        pi.hardware_PWM(buzPin, 331, int(.5e6))
        time.sleep(d)
        pi.hardware_PWM(buzPin, 0, int(.5e6))
            
    if button == 4:
        pi.hardware_PWM(buzPin, 349, int(.5e6))
        time.sleep(d)
        pi.hardware_PWM(buzPin, 0, int(.5e6))

In [ ]:
import sys
# !{sys.executable} -m pip install pynput
import numpy as np
import time
from pynput.keyboard import Key, Controller
from pynput.mouse import Button, Controller as c
import spidev 
import RPi.GPIO as GPIO

# Button/buzzer pins
bright = 19
bup = 20
bleft = 21
bdown = 22
buzPin = 13

pi = pigpio.pi(port = 8887)

# Sets button and buzzer pins
GPIO.setmode(GPIO.BCM)
GPIO.setup(buzPin, GPIO.OUT) 
GPIO.setup(bright, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bup, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bleft, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)
GPIO.setup(bdown, GPIO.IN, pull_up_down = GPIO.PUD_DOWN)

# Create the keyboard and mouse objects
keyboard = Controller()
mouse = c()

# The button layout on the controller, when reading top to bottom you will have
# Up left down right (clockwise)
# ^ > v <
while True:    
    # Press the up arrow
    if GPIO.input(bup):    
        beep(1)
        keyboard.press(Key.up)
        time.sleep(.1)
        keyboard.release(Key.up)
        
    # Right click on the mouse
    if GPIO.input(bright):    
        beep(2)
        mouse.press(Button.right)
        time.sleep(.1)
        mouse.release(Button.right)

    # Left click on the mouse
    if GPIO.input(bdown):
        beep(3)
        mouse.press(Button.left)
        time.sleep(.1)
        mouse.release(Button.left)

    # Press spacebar (usually bound to jump)
    if GPIO.input(bleft):
        beep(4)
        keyboard.press(Key.space)
        time.sleep(.1)       
        keyboard.release(Key.space)
    
    # Left stick function when we almost got it working
    # x_posL, y_posL = getStickL()
    # Get Cartesian for right stick
    x_posR, y_posR = getStickR()
    # Convert to radius for right stick
    rR = radi(x_posR, y_posR)
    # Convert to theta for right stick
    thetaR = angle(x_posR, y_posR)
    # Press the wasd key according to r and theta
    getStickKey(rR, thetaR)
    time.sleep(.1)
    

In [ ]:
# This is a helper block only used in testing, helped us print out the 
# analog stick values and see if we had them properly hooked up.
# This also aided in the cause for determining tolerances and axis 
# for the coordinate system.
to = time.time();

while time.time()<(to+20) or False:
    #r, theta = getStickKey()
    # x_posL, y_posL = getStickL()
    x_posR, y_posR = getStickR()
    #print("Polar Left: ({0:.0f},{1:.0f})".format(radi(x_posL, y_posL),angle(x_posL, y_posL)))
    print("Polar Right: ({0:.0f},{1:.0f})".format(radi(x_posR, y_posR),angle(x_posR, y_posR)))

    # print("Points Left: ({0:.0f},{1:.0f})".format(x_posL,y_posL))
    # print("Points Right: ({0:.0f},{1:.0f})".format(x_posR,y_posR))

    #print(x_pos, y_pos)

    time.sleep(.8)
    
#r, theta = getStickKey()
